# Mining PALSAR Analysis

## Objectives
- Lineament extraction and structural analysis
- Integration with SRTM 30 DEM
- Temporal comparison (2001-2006 vs 2023-2024)

## Study Area
- Coordinates (WGS1984):
  - Point 1: 04°46'0.00"N, 29°34'30.00"E
  - Point 2: 04°46'0.00"N, 29°16'45.00"E
  - Point 3: 05°05'30.00"N, 29°16'45.00"E
  - Point 4: 05°05'30.00"N, 29°34'30.00"E
- Buffer: 20m


In [1]:
pip install geemap earthengine-api numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import required libraries
import geemap
import ee
import numpy as np
from datetime import datetime

# Initialize Earth Engine with Project ID
# IMPORTANT: Replace 'your-project-id' with your actual Google Cloud Project ID
# To find your Project ID:
# 1. Go to https://console.cloud.google.com/
# 2. Click on the project dropdown at the top
# 3. Copy the Project ID (e.g., 'ee-your-username' or 'mineral-exploration-456')

PROJECT_ID = 'ee-okwaretom12'  # <-- REPLACE THIS WITH YOUR PROJECT ID

try:
    # Try to initialize with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized with project: {PROJECT_ID}")
except Exception as e:
    print("Earth Engine not initialized. Starting authentication...")
    print("Please follow the authentication steps:")
    print("1. A browser window will open")
    print("2. Sign in with your Google account")
    print("3. Grant permissions to Earth Engine")
    print("4. Copy the authorization code and paste it when prompted")
    
    # Authenticate (this will open a browser for first-time users)
    ee.Authenticate()
    
    # Initialize after authentication with project ID
    geemap.ee_initialize(project=PROJECT_ID)
    print(f"Earth Engine initialized successfully with project: {PROJECT_ID}!")

# Initialize geemap
Map = geemap.Map()
print("Libraries imported and geemap initialized successfully!")


Earth Engine initialized with project: ee-okwaretom12
Libraries imported and geemap initialized successfully!


## Define Study Area


In [3]:
# Define study area coordinates (WGS1984)
# Convert DMS to decimal degrees
coords = [
    [29.575, 4.766667],  # Point 1: 29°34'30"E, 04°46'0"N
    [29.279167, 4.766667],  # Point 2: 29°16'45"E, 04°46'0"N
    [29.279167, 5.091667],  # Point 3: 29°16'45"E, 05°05'30"N
    [29.575, 5.091667],  # Point 4: 29°34'30"E, 05°05'30"N
]

# Create polygon
study_area = ee.Geometry.Polygon([coords])

# Apply 20m buffer
study_area_buffered = study_area.buffer(20)

# Print study area info (without .getInfo() to avoid blocking)
print("Study area created with coordinates:")
print("  Point 1: 29°34'30\"E, 04°46'0\"N")
print("  Point 2: 29°16'45\"E, 04°46'0\"N")
print("  Point 3: 29°16'45\"E, 05°05'30\"N")
print("  Point 4: 29°34'30\"E, 05°05'30\"N")
print("  Buffer: 20m")

# Add to map
Map.addLayer(study_area_buffered, {'color': 'red'}, 'Study Area (20m buffer)')
Map.centerObject(study_area_buffered, 10)
Map


Study area created with coordinates:
  Point 1: 29°34'30"E, 04°46'0"N
  Point 2: 29°16'45"E, 04°46'0"N
  Point 3: 29°16'45"E, 05°05'30"N
  Point 4: 29°34'30"E, 05°05'30"N
  Buffer: 20m


Map(center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position', 'transparent_b…

## Load SRTM 30 DEM for Lineament Analysis


In [4]:
# Calculate visualization parameters (data-driven ranges)
# These functions calculate optimal min/max values using percentiles (2nd-98th by default)
# to exclude outliers and improve visualization contrast

def calculate_viz_params(image, band_name, study_area, percentile_low=2, percentile_high=98):
    """
    Calculate optimal min/max values for visualization using percentiles
    Excludes outliers for better contrast
    
    Args:
        image: Earth Engine Image
        band_name: Name of the band to visualize
        study_area: Geometry for calculating statistics
        percentile_low: Lower percentile (default: 2)
        percentile_high: Upper percentile (default: 98)
    
    Returns:
        Dictionary with 'min' and 'max' values
    
    Note: Earth Engine returns percentile keys as '{band_name}_p{percentile}'
    """
    # Select the band to ensure we're working with the right band
    band_image = image.select(band_name)
    
    # Calculate percentiles
    stats = band_image.reduceRegion(
        reducer=ee.Reducer.percentile([percentile_low, percentile_high]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    # Earth Engine returns keys as '{band_name}_p{percentile}'
    # Construct the keys with the band name
    min_key = f'{band_name}_p{percentile_low}'
    max_key = f'{band_name}_p{percentile_high}'
    
    return {
        'min': stats.get(min_key),
        'max': stats.get(max_key)
    }

print("Visualization parameter calculation function loaded!")
print("  - calculate_viz_params: For single-band images (2nd-98th percentile)")

# Load SRTM 30m DEM
srtm = ee.Image("USGS/SRTMGL1_003").clip(study_area_buffered)

# Calculate terrain derivatives for lineament detection
elevation = srtm.select('elevation')
slope = ee.Terrain.slope(elevation)
aspect = ee.Terrain.aspect(elevation)
hillshade = ee.Terrain.hillshade(elevation)

# Calculate curvature for enhanced lineament detection
# Use gradient-based approach for better lineament detection
# Smoothing to reduce noise (1-2 pixels optimal: 30-60m at SRTM 30m resolution)
gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
elev_smooth = elevation.convolve(gaussian_filter).resample('bilinear')

# Calculate gradients (first derivatives)
grads = elev_smooth.gradient()
dz_dx = grads.select('x')
dz_dy = grads.select('y')

# Calculate second derivatives (curvature)
second_grads = grads.gradient()
d2z_dx2 = second_grads.select('x')
d2z_dy2 = second_grads.select('y')
total_curvature = d2z_dx2.abs().add(d2z_dy2.abs()).rename('curvature')

# Calculate aspect variance (high variance indicates linear features/lineaments)
# Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
kernel_variance = ee.Kernel.square(radius=2, units='pixels')
aspect_variance = aspect.reduceNeighborhood(
    reducer=ee.Reducer.variance(),
    kernel=kernel_variance
).rename('aspect_variance')

# Calculate data-driven visualization parameters (2nd-98th percentile) for all layers
print("\nCalculating visualization parameters (2nd-98th percentile)...")

elevation_viz = calculate_viz_params(elevation, 'elevation', study_area_buffered, percentile_low=2, percentile_high=98)
slope_viz = calculate_viz_params(slope, 'slope', study_area_buffered, percentile_low=2, percentile_high=98)
aspect_viz = calculate_viz_params(aspect, 'aspect', study_area_buffered, percentile_low=2, percentile_high=98)
hillshade_viz = calculate_viz_params(hillshade, 'hillshade', study_area_buffered, percentile_low=2, percentile_high=98)
curvature_viz = calculate_viz_params(total_curvature, 'curvature', study_area_buffered, percentile_low=2, percentile_high=98)
aspect_var_viz = calculate_viz_params(aspect_variance, 'aspect_variance', study_area_buffered, percentile_low=2, percentile_high=98)

# Get the actual values (handle both server-side and client-side objects)
def get_viz_value(viz_param):
    """Helper to get numeric value from visualization parameter"""
    if hasattr(viz_param, 'getInfo'):
        return float(viz_param.getInfo())
    return float(viz_param) if viz_param is not None else None

# Get min/max values for each layer
elev_min = get_viz_value(elevation_viz['min'])
elev_max = get_viz_value(elevation_viz['max'])
slope_min = get_viz_value(slope_viz['min'])
slope_max = get_viz_value(slope_viz['max'])
aspect_min = get_viz_value(aspect_viz['min'])
aspect_max = get_viz_value(aspect_viz['max'])
hillshade_min = get_viz_value(hillshade_viz['min'])
hillshade_max = get_viz_value(hillshade_viz['max'])
curvature_min = get_viz_value(curvature_viz['min'])
curvature_max = get_viz_value(curvature_viz['max'])
aspect_var_min = get_viz_value(aspect_var_viz['min'])
aspect_var_max = get_viz_value(aspect_var_viz['max'])

# Validate values and provide fallbacks if needed
import math

if elev_min is None or math.isnan(elev_min) or math.isinf(elev_min):
    elev_min = 0.0
if elev_max is None or math.isnan(elev_max) or math.isinf(elev_max):
    elev_max = 2000.0
if elev_min >= elev_max:
    elev_max = elev_min + 100.0

if slope_min is None or math.isnan(slope_min) or math.isinf(slope_min):
    slope_min = 0.0
if slope_max is None or math.isnan(slope_max) or math.isinf(slope_max):
    slope_max = 45.0
if slope_min >= slope_max:
    slope_max = slope_min + 5.0

if aspect_min is None or math.isnan(aspect_min) or math.isinf(aspect_min):
    aspect_min = 0.0
if aspect_max is None or math.isnan(aspect_max) or math.isinf(aspect_max):
    aspect_max = 360.0
if aspect_min >= aspect_max:
    aspect_max = aspect_min + 10.0

if hillshade_min is None or math.isnan(hillshade_min) or math.isinf(hillshade_min):
    hillshade_min = 0.0
if hillshade_max is None or math.isnan(hillshade_max) or math.isinf(hillshade_max):
    hillshade_max = 255.0
if hillshade_min >= hillshade_max:
    hillshade_max = hillshade_min + 10.0

if curvature_min is None or math.isnan(curvature_min) or math.isinf(curvature_min):
    curvature_min = 0.0
if curvature_max is None or math.isnan(curvature_max) or math.isinf(curvature_max):
    curvature_max = 0.03
if curvature_min >= curvature_max:
    curvature_max = curvature_min + 0.001

if aspect_var_min is None or math.isnan(aspect_var_min) or math.isinf(aspect_var_min):
    aspect_var_min = 0.0
if aspect_var_max is None or math.isnan(aspect_var_max) or math.isinf(aspect_var_max):
    aspect_var_max = 50000.0
if aspect_var_min >= aspect_var_max:
    aspect_var_max = aspect_var_min + 1000.0

# Print visualization ranges for reference
print(f"\nVisualization Ranges (2nd-98th percentile):")
print(f"  Elevation: {elev_min:.2f} to {elev_max:.2f} m")
print(f"  Slope: {slope_min:.2f} to {slope_max:.2f} degrees")
print(f"  Aspect: {aspect_min:.2f} to {aspect_max:.2f} degrees")
print(f"  Hillshade: {hillshade_min:.2f} to {hillshade_max:.2f}")
print(f"  Curvature: {curvature_min:.6f} to {curvature_max:.6f}")
print(f"  Aspect Variance: {aspect_var_min:.2f} to {aspect_var_max:.2f}")

# Add to map with data-driven ranges
Map.addLayer(elevation, {
    'min': float(elev_min),
    'max': float(elev_max),
    'palette': ['blue', 'green', 'yellow', 'red']
}, 'SRTM Elevation')

Map.addLayer(slope, {
    'min': float(slope_min),
    'max': float(slope_max),
    'palette': ['white', 'brown']
}, 'Slope')

Map.addLayer(aspect, {
    'min': float(aspect_min),
    'max': float(aspect_max),
    'palette': ['red', 'yellow', 'green', 'cyan', 'blue', 'magenta', 'red']
}, 'Aspect')

Map.addLayer(hillshade, {
    'min': float(hillshade_min),
    'max': float(hillshade_max)
}, 'Hillshade', False)

Map.addLayer(total_curvature, {
    'min': float(curvature_min),
    'max': float(curvature_max),
    'palette': ['white', 'orange', 'red', 'black']
}, 'Curvature (Gradient-based)', False)

Map.addLayer(aspect_variance, {
    'min': float(aspect_var_min),
    'max': float(aspect_var_max),
    'palette': ['white', 'blue', 'purple']
}, 'Aspect Variance', False)

print("\nSRTM DEM and terrain derivatives loaded successfully!")
print("All layers use data-driven visualization ranges (2nd-98th percentile)")
Map

Visualization parameter calculation function loaded!
  - calculate_viz_params: For single-band images (2nd-98th percentile)

Calculating visualization parameters (2nd-98th percentile)...



Visualization Ranges (2nd-98th percentile):
  Elevation: 666.51 to 813.00 m
  Slope: 0.00 to 7.92 degrees
  Aspect: 0.00 to 341.50 degrees
  Hillshade: 163.00 to 198.00
  Curvature: 0.000307 to 0.001654
  Aspect Variance: 832.66 to 21183.13

SRTM DEM and terrain derivatives loaded successfully!
All layers use data-driven visualization ranges (2nd-98th percentile)


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Load PALSAR Data - Period 1 (2001-2006)


In [5]:
def calculate_rgb_viz_params(rgb_image, bands, study_area, percentile_low=2, percentile_high=98):
    """
    Calculate visualization parameters for RGB composite
    Returns min/max that work for all three bands
    """
    stats = rgb_image.select(bands).reduceRegion(
        reducer=ee.Reducer.percentile([percentile_low, percentile_high]),
        geometry=study_area,
        scale=30,
        maxPixels=1e9
    )
    
    first_band = bands[0]
    overall_min = stats.get(f'{first_band}_p{percentile_low}')
    overall_max = stats.get(f'{first_band}_p{percentile_high}')
    
    for band in bands[1:]:
        band_min = stats.get(f'{band}_p{percentile_low}')
        band_max = stats.get(f'{band}_p{percentile_high}')
        overall_min = ee.Number(overall_min).min(ee.Number(band_min))
        overall_max = ee.Number(overall_max).max(ee.Number(band_max))
    
    return {
        'min': overall_min,
        'max': overall_max
    }




In [6]:
# Load PALSAR-2 data for period 2023-2024
# Using ALOS-2 PALSAR-2 yearly SAR_EPOCH collection
palsar_collection_2023_2024 = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH') \
    .filterBounds(study_area_buffered) \
    .filterDate('2023-01-01', '2024-12-31')

print("Checking ALOS-2 PALSAR-2 SAR_EPOCH collection availability for 2023-2024...")
collection_size = palsar_collection_2023_2024.size().getInfo()
print(f"Collection size: {collection_size}")

# Get dates of available images
if collection_size > 0:
    print("\nExtracting dates of available images...")
    
    # Get image dates
    def get_image_dates(image_collection):
        """Extract dates from image collection"""
        dates_list = image_collection.aggregate_array('system:time_start').getInfo()
        # Convert timestamps to readable dates
        from datetime import datetime
        dates = [datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d') for ts in dates_list]
        return sorted(dates)
    
    available_dates = get_image_dates(palsar_collection_2023_2024)
    
    print(f"\nAvailable ALOS-2 PALSAR-2 images ({len(available_dates)} total):")
    for i, date in enumerate(available_dates, 1):
        print(f"  {i}. {date}")
    
    # Get year summary
    years = [date[:4] for date in available_dates]
    year_counts = {}
    for year in years:
        year_counts[year] = year_counts.get(year, 0) + 1
    
    print(f"\nSummary by year:")
    for year in sorted(year_counts.keys()):
        print(f"  {year}: {year_counts[year]} image(s)")
else:
    print("No ALOS-2 PALSAR-2 data found for 2023-2024 period")
    print("Checking for latest available ALOS-2 PALSAR-2 data...")
    
    # Try to get latest available data from ALOS-2
    palsar_collection_latest = ee.ImageCollection('JAXA/ALOS/PALSAR/YEARLY/SAR_EPOCH') \
        .filterBounds(study_area_buffered) \
        .sort('system:time_start', False) \
        .limit(10)
    
    latest_size = palsar_collection_latest.size().getInfo()
    if latest_size > 0:
        print(f"Found {latest_size} latest available ALOS-2 PALSAR-2 image(s):")
        
        def get_image_dates(image_collection):
            """Extract dates from image collection"""
            dates_list = image_collection.aggregate_array('system:time_start').getInfo()
            from datetime import datetime
            dates = [datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d') for ts in dates_list]
            return sorted(dates, reverse=True)  # Latest first
        
        latest_dates = get_image_dates(palsar_collection_latest)
        for i, date in enumerate(latest_dates, 1):
            print(f"  {i}. {date}")
        
        # Use latest available data
        palsar_collection_2023_2024 = palsar_collection_latest
        collection_size = latest_size
    else:
        print("No ALOS-2 PALSAR-2 data available in this collection for the study area")

# Create median composite
palsar_2023_2024 = palsar_collection_2023_2024.median().clip(study_area_buffered)

# Convert to dB scale
palsar_2023_2024_db = palsar_2023_2024.select(['HH', 'HV']).multiply(ee.Image.constant(0.1)).log10().multiply(10)

# Calculate data-driven visualization parameters using the function defined earlier
# Note: calculate_viz_params should be defined in an earlier cell
print("\nCalculating visualization parameters using calculate_viz_params (2nd-98th percentile)...")

palsar_hh_viz_2023 = calculate_viz_params(palsar_2023_2024_db, 'HH', study_area_buffered, percentile_low=2, percentile_high=98)
palsar_hv_viz_2023 = calculate_viz_params(palsar_2023_2024_db, 'HV', study_area_buffered, percentile_low=2, percentile_high=98)

# Get visualization values using the helper function
# Note: get_viz_value should be defined in an earlier cell
palsar_hh_min_2023 = get_viz_value(palsar_hh_viz_2023['min'])
palsar_hh_max_2023 = get_viz_value(palsar_hh_viz_2023['max'])
palsar_hv_min_2023 = get_viz_value(palsar_hv_viz_2023['min'])
palsar_hv_max_2023 = get_viz_value(palsar_hv_viz_2023['max'])

# Validate values and provide fallbacks
import math

if palsar_hh_min_2023 is None or math.isnan(palsar_hh_min_2023) or math.isinf(palsar_hh_min_2023):
    palsar_hh_min_2023 = -25.0
if palsar_hh_max_2023 is None or math.isnan(palsar_hh_max_2023) or math.isinf(palsar_hh_max_2023):
    palsar_hh_max_2023 = 5.0
if palsar_hh_min_2023 >= palsar_hh_max_2023:
    palsar_hh_max_2023 = palsar_hh_min_2023 + 10.0

if palsar_hv_min_2023 is None or math.isnan(palsar_hv_min_2023) or math.isinf(palsar_hv_min_2023):
    palsar_hv_min_2023 = -30.0
if palsar_hv_max_2023 is None or math.isnan(palsar_hv_max_2023) or math.isinf(palsar_hv_max_2023):
    palsar_hv_max_2023 = 0.0
if palsar_hv_min_2023 >= palsar_hv_max_2023:
    palsar_hv_max_2023 = palsar_hv_min_2023 + 10.0

print(f"  HH range: {palsar_hh_min_2023:.2f} to {palsar_hh_max_2023:.2f} dB")
print(f"  HV range: {palsar_hv_min_2023:.2f} to {palsar_hv_max_2023:.2f} dB")

# Add to map with data-driven ranges
Map.addLayer(palsar_2023_2024_db.select('HH'), {
    'min': float(palsar_hh_min_2023),
    'max': float(palsar_hh_max_2023),
    'palette': ['black', 'white']
}, 'PALSAR-2 HH (2023-2024)', False)

Map.addLayer(palsar_2023_2024_db.select('HV'), {
    'min': float(palsar_hv_min_2023),
    'max': float(palsar_hv_max_2023),
    'palette': ['black', 'white']
}, 'PALSAR-2 HV (2023-2024)', False)

# Create RGB composite
hh_hv_ratio_2023 = palsar_2023_2024_db.select('HH').divide(palsar_2023_2024_db.select('HV').add(0.01))
palsar_rgb_2023_2024 = ee.Image.cat([
    palsar_2023_2024_db.select('HH'),
    palsar_2023_2024_db.select('HV'),
    hh_hv_ratio_2023
]).rename(['R', 'G', 'B'])

# Calculate RGB visualization parameters using the function defined earlier
# Note: calculate_rgb_viz_params should be defined in an earlier cell
print("Calculating RGB visualization parameters using calculate_rgb_viz_params...")
palsar_rgb_viz_2023 = calculate_rgb_viz_params(palsar_rgb_2023_2024, ['R', 'G', 'B'], study_area_buffered, percentile_low=2, percentile_high=98)
palsar_rgb_min_2023 = get_viz_value(palsar_rgb_viz_2023['min'])
palsar_rgb_max_2023 = get_viz_value(palsar_rgb_viz_2023['max'])

# Validate RGB values
if palsar_rgb_min_2023 is None or math.isnan(palsar_rgb_min_2023) or math.isinf(palsar_rgb_min_2023):
    palsar_rgb_min_2023 = -25.0
if palsar_rgb_max_2023 is None or math.isnan(palsar_rgb_max_2023) or math.isinf(palsar_rgb_max_2023):
    palsar_rgb_max_2023 = 5.0
if palsar_rgb_min_2023 >= palsar_rgb_max_2023:
    palsar_rgb_max_2023 = palsar_rgb_min_2023 + 10.0

print(f"  RGB range: {palsar_rgb_min_2023:.2f} to {palsar_rgb_max_2023:.2f} dB")

Map.addLayer(palsar_rgb_2023_2024, {
    'min': float(palsar_rgb_min_2023),
    'max': float(palsar_rgb_max_2023)
}, 'PALSAR-2 RGB Composite (2023-2024)')

print(f"\nALOS-2 PALSAR-2 data (2023-2024) loaded successfully!")
print(f"Total images used: {collection_size}")
print("All layers use data-driven visualization ranges (2nd-98th percentile)")
Map

Checking ALOS-2 PALSAR-2 SAR_EPOCH collection availability for 2023-2024...


Collection size: 2

Extracting dates of available images...

Available ALOS-2 PALSAR-2 images (2 total):
  1. 2023-01-01
  2. 2024-01-01

Summary by year:
  2023: 1 image(s)
  2024: 1 image(s)

Calculating visualization parameters using calculate_viz_params (2nd-98th percentile)...
  HH range: 26.09 to 29.47 dB
  HV range: 22.53 to 26.84 dB
Calculating RGB visualization parameters using calculate_rgb_viz_params...
  RGB range: 1.04 to 29.47 dB

ALOS-2 PALSAR-2 data (2023-2024) loaded successfully!
Total images used: 2
All layers use data-driven visualization ranges (2nd-98th percentile)


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [7]:
# Lineament extraction using DEM-based methods
# Method 1: Edge detection on hillshade
def extract_lineaments_dem(elevation_img, slope_img, aspect_img, hillshade_img):
    """Extract lineaments from DEM using multiple methods"""
    
    # Method 1: Slope-based lineaments (steep slopes often indicate faults)
    slope_threshold = slope_img.gt(15)  # Threshold for steep slopes
    
    # Method 2: Aspect-based lineaments (linear features in aspect)
    # Convert aspect to radians and apply edge detection
    aspect_rad = aspect_img.multiply(ee.Image.constant(3.14159)).divide(180)
    
    # Method 3: Hillshade edge detection
    # Apply Sobel filter for edge detection
    # Create kernel for edge detection
    kernel = ee.Kernel.fixed(3, 3, [
        [-1, -1, -1],
        [0, 0, 0],
        [1, 1, 1]
    ])
    
    # Apply convolution for edge detection
    edges_x = hillshade_img.convolve(kernel)
    edges_y = hillshade_img.convolve(kernel.rotate(90))
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Threshold edges to identify lineaments
    lineament_edges = edges.gt(30).rename('Lineament_Edges')
    
    # Method 4: Curvature-based lineaments (gradient approach)
    # Smooth elevation first to reduce noise
    gaussian_filter = ee.Kernel.gaussian(radius=1.5, sigma=1, units='pixels')
    elev_smooth = elevation_img.convolve(gaussian_filter).resample('bilinear')
    
    # Calculate gradients and second derivatives
    grads = elev_smooth.gradient()
    second_grads = grads.gradient()
    d2z_dx2 = second_grads.select('x')
    d2z_dy2 = second_grads.select('y')
    curvature = d2z_dx2.abs().add(d2z_dy2.abs())
    
    # Threshold based on curvature magnitude (adjust threshold as needed)
    lineament_curvature = curvature.gt(0.02).rename('Lineament_Curvature')
    
    # Method 5: Aspect variance lineaments (high variance = linear features)
    # Use reduceNeighborhood with variance reducer (GEE doesn't have focal_variance)
    kernel_variance = ee.Kernel.square(radius=2, units='pixels')
    aspect_var = aspect_img.reduceNeighborhood(
        reducer=ee.Reducer.variance(),
        kernel=kernel_variance
    ).rename('aspect_variance')
    # Use a fixed threshold for aspect variance
    # For aspect (0-360 degrees), variance with 2-pixel radius typically ranges 0-50000
    # High variance (>10000) indicates linear features/lineaments
    # Adjust this threshold based on your study area characteristics
    aspect_var_threshold = ee.Image.constant(10000)  # Adjust as needed (typical range: 5000-20000)
    lineament_aspect_var = aspect_var.gt(aspect_var_threshold).rename('Lineament_Aspect_Var')
    
    # Combine all DEM-based methods
    lineament_combined = lineament_edges.add(lineament_curvature).add(lineament_aspect_var).gt(0).rename('Lineament_DEM')
    
    return {
        'lineament_edges': lineament_edges,
        'lineament_curvature': lineament_curvature,
        'lineament_aspect_var': lineament_aspect_var,
        'lineament_combined': lineament_combined,
        'edges': edges
    }

# Extract lineaments from DEM
dem_lineaments = extract_lineaments_dem(elevation, slope, aspect, hillshade)

# Visualize DEM-based lineaments
Map.addLayer(dem_lineaments['edges'], {
    'min': 0,
    'max': 100,
    'palette': ['black', 'white']
}, 'DEM Edge Detection', False)

Map.addLayer(dem_lineaments['lineament_edges'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from Edges (DEM)')

Map.addLayer(dem_lineaments['lineament_curvature'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'cyan']
}, 'Lineaments from Curvature (DEM)', False)

Map.addLayer(dem_lineaments['lineament_combined'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (DEM)')

print("DEM-based lineaments extracted!")
Map


DEM-based lineaments extracted!


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [8]:
# Lineament extraction from PALSAR radar data
# Radar backscatter can reveal structural features

def extract_lineaments_palsar(palsar_img):
    """Extract lineaments from PALSAR data"""
    
    # Use HH polarization for lineament detection (better for structural features)
    hh = palsar_img.select('HH')
    
    # Apply edge detection using Sobel filter
    kernel_x = ee.Kernel.fixed(3, 3, [
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1]
    ])
    
    kernel_y = ee.Kernel.fixed(3, 3, [
        [-1, -2, -1],
        [0, 0, 0],
        [1, 2, 1]
    ])
    
    # Apply convolution
    edges_x = hh.convolve(kernel_x)
    edges_y = hh.convolve(kernel_y)
    edges = edges_x.pow(2).add(edges_y.pow(2)).sqrt()
    
    # Method 2: HH/HV ratio (enhanced lineament detection)
    # Ratio helps identify structural features
    hh_hv_ratio = palsar_img.select('HH').divide(palsar_img.select('HV').add(0.01)).rename('HH_HV_Ratio')
    
    # Edge detection on ratio
    ratio_edges_x = hh_hv_ratio.convolve(kernel_x)
    ratio_edges_y = hh_hv_ratio.convolve(kernel_y)
    ratio_edges = ratio_edges_x.pow(2).add(ratio_edges_y.pow(2)).sqrt()
    
    # Combine HH edges and ratio edges for better detection
    combined_edges = edges.add(ratio_edges).rename('edges')
    
    # Threshold to identify lineaments using data-driven threshold
    # Calculate threshold based on 75th percentile of edge values
    # Get the actual band name first
    edge_band_name = combined_edges.bandNames().get(0).getInfo()
    
    edge_stats = combined_edges.reduceRegion(
        reducer=ee.Reducer.percentile([75]),
        geometry=study_area_buffered,
        scale=30,
        maxPixels=1e9
    ).getInfo()
    
    # Construct the correct key
    threshold_key = f'{edge_band_name}_p75'
    edge_threshold_val = edge_stats.get(threshold_key)
    
    # Use fallback if key not found
    if edge_threshold_val is None:
        # Try alternative key formats
        edge_threshold_val = edge_stats.get('edges_p75') or edge_stats.get('p75')
        if edge_threshold_val is None:
            # Use a default threshold (20 is typical for PALSAR edges)
            edge_threshold_val = 20.0
            print(f"Warning: Could not find threshold key, using default: {edge_threshold_val}")
    
    edge_threshold = ee.Number(edge_threshold_val)
    lineament_threshold = combined_edges.gt(edge_threshold).rename('Lineament_PALSAR')
    
    return {
        'edges': combined_edges,
        'lineament_threshold': lineament_threshold,
        'hh_hv_ratio': hh_hv_ratio.rename('hh_hv_ratio')
    }

# Extract lineaments from 2023-2024 period only
palsar_lineaments_2023_2024 = extract_lineaments_palsar(palsar_2023_2024_db)

# Calculate data-driven visualization parameters (2nd-98th percentile)
print("Calculating visualization parameters for PALSAR lineaments (2nd-98th percentile)...")

# Edges visualization parameters
edges_viz_2023 = calculate_viz_params(palsar_lineaments_2023_2024['edges'], 'edges', study_area_buffered, percentile_low=2, percentile_high=98)
edges_min_2023 = get_viz_value(edges_viz_2023['min'])
edges_max_2023 = get_viz_value(edges_viz_2023['max'])

# HH/HV ratio visualization parameters
hh_hv_viz_2023 = calculate_viz_params(palsar_lineaments_2023_2024['hh_hv_ratio'], 'hh_hv_ratio', study_area_buffered, percentile_low=2, percentile_high=98)
hh_hv_min_2023 = get_viz_value(hh_hv_viz_2023['min'])
hh_hv_max_2023 = get_viz_value(hh_hv_viz_2023['max'])

# Validate values and provide fallbacks
import math

if edges_min_2023 is None or math.isnan(edges_min_2023) or math.isinf(edges_min_2023):
    edges_min_2023 = 0.0
if edges_max_2023 is None or math.isnan(edges_max_2023) or math.isinf(edges_max_2023):
    edges_max_2023 = 50.0
if edges_min_2023 >= edges_max_2023:
    edges_max_2023 = edges_min_2023 + 10.0

if hh_hv_min_2023 is None or math.isnan(hh_hv_min_2023) or math.isinf(hh_hv_min_2023):
    hh_hv_min_2023 = 0.0
if hh_hv_max_2023 is None or math.isnan(hh_hv_max_2023) or math.isinf(hh_hv_max_2023):
    hh_hv_max_2023 = 5.0
if hh_hv_min_2023 >= hh_hv_max_2023:
    hh_hv_max_2023 = hh_hv_min_2023 + 1.0

print(f"  Edges range: {edges_min_2023:.2f} to {edges_max_2023:.2f}")
print(f"  HH/HV Ratio range: {hh_hv_min_2023:.2f} to {hh_hv_max_2023:.2f}")

# Visualize PALSAR-based lineaments with data-driven ranges
Map.addLayer(palsar_lineaments_2023_2024['edges'], {
    'min': float(edges_min_2023),
    'max': float(edges_max_2023),
    'palette': ['black', 'white']
}, 'PALSAR Edges (2023-2024)', False)

# Lineament thresholds (binary, so fixed 0-1 range is appropriate)
Map.addLayer(palsar_lineaments_2023_2024['lineament_threshold'], {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Lineaments from PALSAR (2023-2024)')

# Visualize HH/HV ratio for structural analysis with data-driven ranges
Map.addLayer(palsar_lineaments_2023_2024['hh_hv_ratio'], {
    'min': float(hh_hv_min_2023),
    'max': float(hh_hv_max_2023),
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'HH/HV Ratio (2023-2024)', False)

print("\nPALSAR-based lineaments extracted for 2023-2024!")
print("All visualizations use data-driven ranges (2nd-98th percentile)")
Map

Calculating visualization parameters for PALSAR lineaments (2nd-98th percentile)...
  Edges range: 0.70 to 8.44
  HH/HV Ratio range: 1.04 to 1.20

PALSAR-based lineaments extracted for 2023-2024!
All visualizations use data-driven ranges (2nd-98th percentile)


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

In [10]:
# Combine DEM and PALSAR lineaments for comprehensive lineament map
# Enhanced approach: DEM curvature + PALSAR edges/ratios + aspect variance
def combine_lineaments(dem_lineaments, palsar_lineaments, aspect_variance_img):
    """Combine DEM and PALSAR lineaments with enhanced confidence"""
    
    # Extract individual components
    dem_curvature = dem_lineaments['lineament_curvature'].unmask(0)
    dem_aspect_var = dem_lineaments['lineament_aspect_var'].unmask(0)
    dem_edges = dem_lineaments['lineament_edges'].unmask(0)
    palsar_edges = palsar_lineaments['edges'].unmask(0)
    palsar_threshold = palsar_lineaments['lineament_threshold'].unmask(0)
    
    # Normalize PALSAR edges to 0-1 range using unitScale (server-side operation)
    # Use percentile-based scaling to avoid outliers
    # PALSAR edges typically range from -50 to 50 for edge detection
    palsar_edges_norm = palsar_edges.unitScale(-50, 50).clamp(0, 1)
    
    # Normalize aspect variance to 0-1 range using unitScale
    # Aspect variance typically ranges from 0 to ~50000 for 2-pixel radius
    aspect_var_norm = aspect_variance_img.unitScale(0, 50000).clamp(0, 1)
    
    # Combine with weights for higher confidence detection
    # DEM curvature: 0.3, PALSAR edges: 0.3, Aspect variance: 0.2, DEM edges: 0.2
    combined_score = (
        dem_curvature.multiply(0.3)
        .add(palsar_edges_norm.multiply(0.3))
        .add(aspect_var_norm.multiply(0.2))
        .add(dem_edges.multiply(0.2))
    )
    
    # Threshold for final lineament map (adjust threshold as needed)
    combined = combined_score.gt(0.4).rename('Combined_Lineaments')
    
    # Create confidence map (0-1 scale)
    confidence = combined_score.rename('Lineament_Confidence')
    
    return combined.addBands(confidence)

# Create combined lineament map for 2023-2024 period only
# Pass aspect_variance from Cell 6
combined_lineaments_2023_2024 = combine_lineaments(
    dem_lineaments,
    palsar_lineaments_2023_2024,
    aspect_variance
)

# Calculate data-driven visualization parameters for confidence map (2nd-98th percentile)
print("Calculating visualization parameters for combined lineaments...")
confidence_viz_2023 = calculate_viz_params(
    combined_lineaments_2023_2024, 
    'Lineament_Confidence', 
    study_area_buffered, 
    percentile_low=2, 
    percentile_high=98
)
confidence_min_2023 = get_viz_value(confidence_viz_2023['min'])
confidence_max_2023 = get_viz_value(confidence_viz_2023['max'])

# Validate confidence values
import math
if confidence_min_2023 is None or math.isnan(confidence_min_2023) or math.isinf(confidence_min_2023):
    confidence_min_2023 = 0.0
if confidence_max_2023 is None or math.isnan(confidence_max_2023) or math.isinf(confidence_max_2023):
    confidence_max_2023 = 1.0
if confidence_min_2023 >= confidence_max_2023:
    confidence_max_2023 = confidence_min_2023 + 0.1

print(f"  Confidence range: {confidence_min_2023:.4f} to {confidence_max_2023:.4f}")

# Visualize combined lineaments
# Binary lineament map (fixed 0-1 range is appropriate for binary data)
Map.addLayer(combined_lineaments_2023_2024.select('Combined_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'red']
}, 'Combined Lineaments (2023-2024)')

# Confidence map with data-driven range
Map.addLayer(combined_lineaments_2023_2024.select('Lineament_Confidence'), {
    'min': float(confidence_min_2023),
    'max': float(confidence_max_2023),
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Confidence (2023-2024)', False)

print("\nCombined lineament map created for 2023-2024!")
print("Confidence map uses data-driven visualization range (2nd-98th percentile)")
Map

Calculating visualization parameters for combined lineaments...


  Confidence range: 0.1623 to 0.4277

Combined lineament map created for 2023-2024!
Confidence map uses data-driven visualization range (2nd-98th percentile)


Map(bottom=127778.0, center=[4.929170082547151, 29.42708350000096], controls=(WidgetControl(options=['position…

## Lineament Structural Complexity Analysis


In [11]:
# Lineament Structural Complexity Analysis
# Using Lineament Confidence Map as proxy for density/structural complexity
# The confidence map (0-1 scale) shows where lineaments are most concentrated

print("=" * 60)
print("LINEAMENT STRUCTURAL COMPLEXITY ANALYSIS (2023-2024)")
print("Using Lineament Confidence Map (proxy for density)")
print("=" * 60)

# Get comprehensive confidence statistics
print("Calculating confidence statistics...")
confidence_stats = combined_lineaments_2023_2024.select('Lineament_Confidence').reduceRegion(
    reducer=ee.Reducer.minMax().combine(
        ee.Reducer.mean().combine(
            ee.Reducer.median().combine(
                ee.Reducer.percentile([10, 25, 50, 75, 90, 95]),
                sharedInputs=True
            ),
            sharedInputs=True
        ),
        sharedInputs=True
    ),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo()

# Extract statistics
conf_min = confidence_stats.get('Lineament_Confidence_min', 0)
conf_max = confidence_stats.get('Lineament_Confidence_max', 0)
conf_mean = confidence_stats.get('Lineament_Confidence_mean', 0)
conf_median = confidence_stats.get('Lineament_Confidence_median', 0)
conf_p10 = confidence_stats.get('Lineament_Confidence_p10', 0)
conf_p25 = confidence_stats.get('Lineament_Confidence_p25', 0)
conf_p50 = confidence_stats.get('Lineament_Confidence_p50', 0)
conf_p75 = confidence_stats.get('Lineament_Confidence_p75', 0)
conf_p90 = confidence_stats.get('Lineament_Confidence_p90', 0)
conf_p95 = confidence_stats.get('Lineament_Confidence_p95', 0)

print("\n" + "=" * 60)
print("LINEAMENT CONFIDENCE STATISTICS (2023-2024)")
print("=" * 60)
print(f"Minimum: {conf_min:.4f}")
print(f"10th percentile: {conf_p10:.4f}")
print(f"25th percentile: {conf_p25:.4f}")
print(f"Median (50th percentile): {conf_median:.4f}")
print(f"Mean: {conf_mean:.4f}")
print(f"75th percentile: {conf_p75:.4f}")
print(f"90th percentile: {conf_p90:.4f}")
print(f"95th percentile: {conf_p95:.4f}")
print(f"Maximum: {conf_max:.4f}")
print("=" * 60)
print("Note: Higher confidence = higher structural complexity")
print("=" * 60)

# Classify into structural complexity zones based on percentiles
print("\nClassifying structural complexity zones...")

very_low = combined_lineaments_2023_2024.select('Lineament_Confidence').lt(conf_p25).rename('Very_Low')
low = combined_lineaments_2023_2024.select('Lineament_Confidence').gte(conf_p25).And(
    combined_lineaments_2023_2024.select('Lineament_Confidence').lt(conf_p50)
).rename('Low')
medium = combined_lineaments_2023_2024.select('Lineament_Confidence').gte(conf_p50).And(
    combined_lineaments_2023_2024.select('Lineament_Confidence').lt(conf_p75)
).rename('Medium')
high = combined_lineaments_2023_2024.select('Lineament_Confidence').gte(conf_p75).And(
    combined_lineaments_2023_2024.select('Lineament_Confidence').lt(conf_p90)
).rename('High')
very_high = combined_lineaments_2023_2024.select('Lineament_Confidence').gte(conf_p90).rename('Very_High')

# Calculate pixel counts for each class
total_pixels = combined_lineaments_2023_2024.select('Lineament_Confidence').reduceRegion(
    reducer=ee.Reducer.count(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('Lineament_Confidence', 1)

very_low_count = very_low.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('Very_Low', 0)

low_count = low.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('Low', 0)

medium_count = medium.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('Medium', 0)

high_count = high.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('High', 0)

very_high_count = very_high.reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=study_area_buffered,
    scale=30,
    maxPixels=1e9
).getInfo().get('Very_High', 0)

# Calculate percentages
very_low_pct = (very_low_count / total_pixels * 100) if total_pixels > 0 else 0
low_pct = (low_count / total_pixels * 100) if total_pixels > 0 else 0
medium_pct = (medium_count / total_pixels * 100) if total_pixels > 0 else 0
high_pct = (high_count / total_pixels * 100) if total_pixels > 0 else 0
very_high_pct = (very_high_count / total_pixels * 100) if total_pixels > 0 else 0

print("\n" + "=" * 60)
print("STRUCTURAL COMPLEXITY CLASS DISTRIBUTION")
print("=" * 60)
print(f"Very Low Complexity (< {conf_p25:.4f}): {very_low_pct:.2f}%")
print(f"Low Complexity ({conf_p25:.4f} - {conf_p50:.4f}): {low_pct:.2f}%")
print(f"Medium Complexity ({conf_p50:.4f} - {conf_p75:.4f}): {medium_pct:.2f}%")
print(f"High Complexity ({conf_p75:.4f} - {conf_p90:.4f}): {high_pct:.2f}%")
print(f"Very High Complexity (≥ {conf_p90:.4f}): {very_high_pct:.2f}%")
print("=" * 60)
print(f"Total pixels analyzed: {total_pixels:,}")
print("=" * 60)

# Calculate data-driven visualization parameters (2nd-98th percentile)
print("\nCalculating visualization parameters...")
confidence_viz_2023 = calculate_viz_params(
    combined_lineaments_2023_2024, 
    'Lineament_Confidence', 
    study_area_buffered, 
    percentile_low=2, 
    percentile_high=98
)
confidence_min_2023 = get_viz_value(confidence_viz_2023['min'])
confidence_max_2023 = get_viz_value(confidence_viz_2023['max'])

# Validate values
import math
if confidence_min_2023 is None or math.isnan(confidence_min_2023) or math.isinf(confidence_min_2023):
    confidence_min_2023 = 0.0
if confidence_max_2023 is None or math.isnan(confidence_max_2023) or math.isinf(confidence_max_2023):
    confidence_max_2023 = 1.0
if confidence_min_2023 >= confidence_max_2023:
    confidence_max_2023 = confidence_min_2023 + 0.1

print(f"  Visualization range: {confidence_min_2023:.4f} to {confidence_max_2023:.4f}")

# Visualize continuous confidence map with data-driven range
Map.addLayer(combined_lineaments_2023_2024.select('Lineament_Confidence'), {
    'min': float(confidence_min_2023),
    'max': float(confidence_max_2023),
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}, 'Lineament Confidence / Structural Complexity (2023-2024)')

# Visualize complexity zones
Map.addLayer(very_high, {
    'min': 0,
    'max': 1,
    'palette': ['white', 'darkred']
}, 'Very High Complexity (Top 10%)', False)

Map.addLayer(high, {
    'min': 0,
    'max': 1,
    'palette': ['white', 'red']
}, 'High Complexity (75th-90th percentile)', False)

Map.addLayer(medium, {
    'min': 0,
    'max': 1,
    'palette': ['white', "orange"]
}, 'Medium Complexity (50th-75th percentile)', False)

Map.addLayer(low, {
    'min': 0,
    'max': 1,
    'palette': ['white', "yellow"]
}, 'Low Complexity (25th-50th percentile)', False)

Map.addLayer(very_low, {
    'min': 0,
    'max': 1,
    'palette': ['white', 'lightblue']
}, 'Very Low Complexity (Bottom 25%)', False)

# Also show binary lineaments for reference
Map.addLayer(combined_lineaments_2023_2024.select('Combined_Lineaments'), {
    'min': 0,
    'max': 1,
    'palette': ['black', 'yellow']
}, 'Combined Lineaments (Binary)', False)

print("\n" + "=" * 60)
print("STRUCTURAL COMPLEXITY ANALYSIS COMPLETE")
print("=" * 60)
print("Interpretation:")
print("  🔴 Very High Complexity (Top 10%): Highest structural complexity")
print("     - Most favorable for mineralization")
print("     - Multiple intersecting lineaments")
print("     - Highest priority exploration targets")
print("")
print("  🔴 High Complexity (75th-90th percentile): High structural complexity")
print("     - Good potential for mineralization")
print("     - Secondary exploration targets")
print("")
print("  🟠 Medium Complexity (50th-75th percentile): Moderate complexity")
print("     - Some structural features present")
print("     - Moderate exploration potential")
print("")
print("  🟡 Low Complexity (25th-50th percentile): Low complexity")
print("     - Few structural features")
print("     - Lower exploration priority")
print("")
print("  🔵 Very Low Complexity (Bottom 25%): Minimal complexity")
print("     - Uniform terrain, few lineaments")
print("     - Lowest exploration priority")
print("=" * 60)
print("Note: This confidence map serves as a proxy for lineament density")
print("Higher confidence = higher lineament concentration = higher complexity")
print("=" * 60)

Map

LINEAMENT STRUCTURAL COMPLEXITY ANALYSIS (2023-2024)
Using Lineament Confidence Map (proxy for density)
Calculating confidence statistics...



LINEAMENT CONFIDENCE STATISTICS (2023-2024)
Minimum: 0.1513
10th percentile: 0.1700
25th percentile: 0.1816
Median (50th percentile): 0.2012
Mean: 0.2309
75th percentile: 0.2285
90th percentile: 0.3926
95th percentile: 0.4121
Maximum: 0.7581
Note: Higher confidence = higher structural complexity

Classifying structural complexity zones...

STRUCTURAL COMPLEXITY CLASS DISTRIBUTION
Very Low Complexity (< 0.1816): 24.52%
Low Complexity (0.1816 - 0.2012): 24.62%
Medium Complexity (0.2012 - 0.2285): 24.70%
High Complexity (0.2285 - 0.3926): 15.79%
Very High Complexity (≥ 0.3926): 9.80%
Total pixels analyzed: 1,325,286

Calculating visualization parameters...
  Visualization range: 0.1623 to 0.4277

STRUCTURAL COMPLEXITY ANALYSIS COMPLETE
Interpretation:
  🔴 Very High Complexity (Top 10%): Highest structural complexity
     - Most favorable for mineralization
     - Multiple intersecting lineaments
     - Highest priority exploration targets

  🔴 High Complexity (75th-90th percentile): High

Map(bottom=127778.0, center=[4.92951505644009, 29.42825317382813], controls=(WidgetControl(options=['position'…

## Lineament Orientation Analysis


In [13]:
# Analyze lineament orientations (rose diagram data)
# Extract lineament directions from aspect and edge orientations

def analyze_lineament_orientation(lineament_img, aspect_img):
    """Analyze lineament orientations"""
    
    # Use aspect where lineaments exist
    lineament_aspect = aspect_img.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    # Classify into orientation classes (N, NE, E, SE, S, SW, W, NW)
    # Convert to 8 classes (45-degree intervals)
    orientation_classes = lineament_aspect.divide(45).int().mod(8).rename('Orientation_Class')
    
    # Create orientation map
    orientation_map = orientation_classes.updateMask(lineament_img.select('Combined_Lineaments').gt(0))
    
    return orientation_map

# Analyze orientations for 2023-2024 period only
# Using combined DEM with 2023-2024 PALSAR-2 data
lineament_orientation_2023_2024 = analyze_lineament_orientation(
    combined_lineaments_2023_2024,
    aspect
)

# Visualize orientations
# Orientation classes are discrete (0-7), so fixed range is appropriate
Map.addLayer(lineament_orientation_2023_2024, {
    'min': 0,
    'max': 7,
    'palette': ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'magenta']
}, 'Lineament Orientation (2023-2024)')

print("Lineament orientation analysis completed for 2023-2024!")
print("Note: 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW")
print("Based on combined DEM + PALSAR-2 (2023-2024) lineaments")
Map

Lineament orientation analysis completed for 2023-2024!
Note: 0=N, 1=NE, 2=E, 3=SE, 4=S, 5=SW, 6=W, 7=NW
Based on combined DEM + PALSAR-2 (2023-2024) lineaments


Map(bottom=127778.0, center=[4.92951505644009, 29.42825317382813], controls=(WidgetControl(options=['position'…

In [ ]:
# Export lineament maps
# Note: Uncomment and run to export to Google Drive

def export_lineament_maps():
    """Export all lineament maps"""
    
    # DEM-based lineaments
    geemap.ee_export_image(
        dem_lineaments['lineament_combined'],
        filename='lineaments_dem.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # PALSAR-based lineaments
    geemap.ee_export_image(
        palsar_lineaments_2001_2006['lineament_threshold'],
        filename='lineaments_palsar_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        palsar_lineaments_2023_2024['lineament_threshold'],
        filename='lineaments_palsar_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Combined lineaments
    geemap.ee_export_image(
        combined_lineaments_2001_2006,
        filename='lineaments_combined_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        combined_lineaments_2023_2024,
        filename='lineaments_combined_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Lineament density
    geemap.ee_export_image(
        lineament_density_2001_2006,
        filename='lineament_density_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        lineament_density_2023_2024,
        filename='lineament_density_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Lineament orientation
    geemap.ee_export_image(
        lineament_orientation_2001_2006,
        filename='lineament_orientation_2001_2006.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    geemap.ee_export_image(
        lineament_orientation_2023_2024,
        filename='lineament_orientation_2023_2024.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )
    
    # Change detection
    geemap.ee_export_image(
        lineament_changes,
        filename='lineament_changes.tif',
        scale=30,
        region=study_area_buffered,
        file_per_band=False
    )

print("Export functions defined. Uncomment and call to export maps.")
print("Example: export_lineament_maps()")


Export functions defined. Uncomment and call to export maps.
Example: export_lineament_maps()
